In [1]:
# biblioteca de automacao web

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# biblioteca para minerar html

from bs4 import BeautifulSoup

# biblioteca para manusear dados
import pandas as pd
import geopandas as gpd

# bibliotecas de geolocalização
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Point

# bibliotecas para contruir os mapas 
import folium
from folium.plugins import FastMarkerCluster

# Mineração dos Jogos

Peguei as informações do site Meu Timão

In [8]:
# abrir (Depois de aberto é necessário instalar o adblock manualmente)

driver = webdriver.Chrome()

In [9]:
df = []
erros = []
link_partida = {}
page_inicial = 1
page_final = 205

for num in range(page_inicial, page_final):
    
    # entra no site e pega todos resultados dos jogos
    driver.get(f'https://www.meutimao.com.br/resultados-dos-jogos-do-corinthians/?pag={num}')
    site = driver.page_source
    soup = BeautifulSoup(site, 'html.parser')
    jogos_lista = soup.find(id='jogos_lista')
    jogos = jogos_lista.find_all('li')
    

    
    # estabelece ou limpa a lista
    todos_links = []
    
    # looping de mineração de cada jogo     
    for jogo in jogos:
        
        # pega o link com as informações completas e coloca na lista
        link = jogo.find('a').get('href')
        todos_links.append('https://www.meutimao.com.br' + link)
        
        # marcação do que deve ser pego no texto da partida
        marcacao_partida = jogo.find('a').text.find(':')
        
        # texto da partida
        partida = jogo.find('a').text[:marcacao_partida].replace('\n', '').split('x')
        
        # coloca o dicionário para pegar o nome da partida caso tenha erro
        link_partida['https://www.meutimao.com.br' + link] = jogo.find('a').text[:marcacao_partida].replace('\n', '')
    
    # looping para anlisar os detalhes dos links pegos
    for link_timao in todos_links:
        
        # tenta pegas as informações
        try:
            
            # entra no link
            driver.get(link_timao)
            
            # pega o html e processa
            resultado = driver.page_source
            soup_resultado = BeautifulSoup(resultado, 'html.parser')
            
            # pega as informações da caixa
            h2 = soup_resultado.find(id="container_placar").find_all('h2')[1]

            # pega o campeonato
            campeonato = h2.text.strip().split('\n')[0]
            
            # pega a data
            data = h2.text.strip().split('\n')[1]
            
            # pega o placar e divide entre mandante e visitante
            placar_completo = h2.text.strip().split('\n')[2].strip()
            placar = placar_completo.split(' x ')
            
            # pega informações do time mandante (nome e gols)
            marcacao_mandante = placar[0].strip().find(' ', -1)
            mandante = placar[0].strip()[:marcacao_mandante].strip()
            gol_mandante = int(placar[0].strip()[marcacao_mandante:].strip())

            # pega informações do time visitante (nome e gols)
            marcacao_visitante = placar[1].strip().find(' ')
            visitante = placar[1].strip()[marcacao_visitante:].strip()
            gol_visitante = int(placar[1].strip()[:marcacao_visitante].strip())

            # pega o local
            local = h2.text.strip().split('\n')[4]
            
            # coloca as informações em uma lista
            df.append([campeonato, data, mandante, gol_mandante, gol_visitante, visitante, local])

        except:
            
            # caso tenha erro, pega o número da página, o link da partida e a partida
            erros.append([num, link_timao, link_partida['https://www.meutimao.com.br' + link]])
            
print('FIM')

FIM


In [10]:
# transforma a lista em um Data Frame
jogos_corinthians_masculino = pd.DataFrame(df, columns=['campeonato', 'data', 'mandante', 'gol mandante', 'gol visitante', 'visitante', 'local'])

# exporta arquivo para csv
jogos_corinthians_masculino.to_csv('jogos_corinthians_masculino_20230703.csv', index=False)

In [11]:
df_erro = pd.DataFrame(erros, columns=['numero da pagina', 'link', 'partida'])

In [12]:
df_erro.to_csv('erros_20230703.csv', index=False)

# Verificar os erros

In [13]:
# subindo csv que não conseguiu pegar

nao_encontrados = pd.read_csv('erros_20230703.csv')

In [14]:
nao_encontrados.head()

,numero da pagina,link,partida
0,20,https://www.meutimao.com.br/jogo/496/florida_c...,Corinthians 1 x 0 Grêmio
1,21,https://www.meutimao.com.br/jogo/475/brasileir...,Corinthians 3 x 0 Bahia
2,21,https://www.meutimao.com.br/jogo/472/brasileir...,Corinthians 3 x 0 Bahia
3,21,https://www.meutimao.com.br/jogo/471/brasileir...,Corinthians 3 x 0 Bahia
4,21,https://www.meutimao.com.br/jogo/468/brasileir...,Corinthians 3 x 0 Bahia


In [ ]:
# abrir (Depois de aberto é necessário instalar o adblock manualmente)

driver = webdriver.Chrome()

In [15]:
for index, coluna in nao_encontrados.iterrows():
    driver.get(f'https://www.meutimao.com.br/resultados-dos-jogos-do-corinthians/?pag={coluna["numero da pagina"]}')
    site = driver.page_source
    soup = BeautifulSoup(site, 'html.parser')
    jogos_lista = soup.find(id='jogos_lista')
    jogos = jogos_lista.find_all('li')
    
    for jogo in jogos:
        link = jogo.find('a').get('href')
        if 'https://www.meutimao.com.br' + link == coluna['link']:
            
            partida_completa = jogo.find('a').text
            pontos = partida_completa.find(':')
            partida = partida_completa[:pontos].replace('\n', '')

            marcacao_mandante = partida.split(' x ')[0].find(' ', -1)
            mandante = partida.split(' x ')[0][:marcacao_mandante].strip()
            gol_mandante = partida.split(' x ')[0][marcacao_mandante:].strip()

            marcacao_visitante = partida.split(' x ')[1].find(' ')
            visitante = partida.split(' x ')[1][marcacao_visitante:].strip()
            gol_visitante = partida.split(' x ')[1][:marcacao_visitante].strip()
            data = jogo.time['datetime']
            estadio = jogo.p.text.strip()
            campeonato = jogo.find_all('p')[1].text.strip()
            df.append([campeonato, data, mandante, gol_mandante, gol_visitante, visitante, estadio])

            

In [16]:
df_erros = pd.DataFrame(df, columns=list(jogos_corinthians_masculino.columns))

In [17]:
jogos_completos = jogos_corinthians_masculino.merge(df_erros, how='outer')

In [18]:
jogos_completos.to_csv('jogos_corinthians_masculino_completo_20230703.csv', index=False)

# Incluir localização

In [21]:
len(jogos_completos)

6093

In [24]:
df_estadios = jogos_completos[['local']].drop_duplicates().reset_index(drop=True)

# instanciando a biblioteca que tratá as informações de geolocalização
locator = Nominatim(user_agent='MyGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

# aplicando a geolocalização
df_estadios['location'] = df_estadios['local'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Municipal de Campo Grande, Campo Grande, MS.',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/u

RateLimiter swallowed an error after 2 retries. Called with (*('Municipal de Campo Grande, Campo Grande, MS.',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Estádio de Halmstad, Halmstad, Suécia.',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Vila Sophia, São Paulo, SP.',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10

RateLimiter swallowed an error after 2 retries. Called with (*('Vila Sophia, São Paulo, SP.',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10/ss

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Estádio da Rua 32',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10/ssl.py", 

In [25]:
# inlcuindo as informações de geolocalização (ponto, endereço, tipo, classe, latitude e longitude

df_estadios['ponto'] = df_estadios['location'].apply(lambda x : x.point if x else None)
df_estadios['endereco'] = df_estadios['location'].apply(lambda x : x.address if x else None)
df_estadios['tipo'] = df_estadios['location'].apply(lambda x : x.raw['type'] if x else None)
df_estadios['classe'] = df_estadios['location'].apply(lambda x : x.raw['class'] if x else None)
df_estadios['latitude'] = df_estadios['location'].apply(lambda x : x.latitude if x else None)
df_estadios['longitude'] = df_estadios['location'].apply(lambda x : x.longitude if x else None)

In [26]:
df_estadios['tipo'].drop_duplicates().values

array(['stadium', 'administrative', 'neighbourhood', None, 'yes',
       'pharmacy', 'sports_centre', 'suburb', 'park', 'residential',
       'hotel', 'restaurant', 'stationery', 'locality', 'library', 'deli',
       'hamlet', 'parking', 'station', 'car', 'recreation_ground',
       'bus_stop', 'water', 'secondary', 'pitch', 'grandstand', 'company',
       'educational_institution', 'sports_hall', 'platform', 'attraction',
       'car_rental', 'townhall', 'construction', 'sports', 'ticket',
       'bicycle_rental', 'bar', 'cemetery', 'wood', 'trunk', 'tertiary',
       'protected_area', 'primary', 'peak', 'grave_yard', 'village',
       'shoes', 'square', 'fuel', 'living_street', 'supermarket',
       'bus_station', 'river', 'marketplace', 'kiosk', 'apartments'],
      dtype=object)

In [27]:
# aqui estão os locais relacionados a esporte

df_estadios_esporte = df_estadios.query('tipo == "stadium" or tipo == "sports_centre" or tipo == "pitch" or tipo == "sports_hall" or tipo == "sports"')

In [28]:
df_estadios_esporte.head()

,local,location,ponto,endereco,tipo,classe,latitude,longitude
0,"Neo Química Arena, São Paulo, SP.","(Neo Química Arena, 111, Caminho Areita Calfat...","23 32m 43.0562s S, 46 28m 27.4312s W","Neo Química Arena, 111, Caminho Areita Calfat ...",stadium,leisure,-23.545293,-46.474286
1,"Arena da Baixada, Curitiba, PR.","(Arena da Baixada, 1260, Rua Buenos Aires, Águ...","25 26m 54.1939s S, 49 16m 34.3796s W","Arena da Baixada, 1260, Rua Buenos Aires, Água...",stadium,building,-25.448387,-49.276217
3,"Estádio Banco Guayaquil, Quito, Equador.","(Estadio Banco Guayaquil, Manuel Rodriguez de ...","0 20m 57.5587s S, 78 28m 14.5417s W","Estadio Banco Guayaquil, Manuel Rodriguez de Q...",stadium,leisure,-0.349322,-78.470706
5,"Estádio Diego Armando Maradona, Buenos Aires, ...","(Estadio Diego Armando Maradona, 2151, Gavilán...","34 36m 21.8799s S, 58 28m 21.5999s W","Estadio Diego Armando Maradona, 2151, Gavilán,...",stadium,leisure,-34.606078,-58.472667
6,"Maracanã, Rio de Janeiro, RJ.","(Estádio Jornalista Mário Filho, Avenida Rei P...","22 54m 43.3256s S, 43 13m 52.1629s W","Estádio Jornalista Mário Filho, Avenida Rei Pe...",stadium,leisure,-22.912035,-43.231156


In [30]:
# buscando pelo nome do local + estádio

df_estadio_estadio = pd.DataFrame()

df_estadio_estadio['estadio'] = df_estadios.query('tipo != "stadium" & tipo != "sports_centre" & tipo != "pitch" & tipo != "sports_hall" & tipo != "sports"')[['local']].apply(lambda x : 'Estadio ' + x)
df_estadio_estadio['local'] = df_estadios.query('tipo != "stadium" & tipo != "sports_centre" & tipo != "pitch" & tipo != "sports_hall" & tipo != "sports"')[['local']]
df_estadio_estadio.reset_index(drop=True, inplace=True)

In [32]:
# instanciando a biblioteca que tratá as informações de geolocalização
locator = Nominatim(user_agent='MyGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

# aplicando a geolocalização
df_estadio_estadio['location'] = df_estadio_estadio['estadio'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Estadio Rua Tabor, São Paulo, SP.',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/pyth

RateLimiter swallowed an error after 2 retries. Called with (*('Estadio Rua Tabor, São Paulo, SP.',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Estadio Estádio da Rua 32',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1374, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10/s

In [33]:
# inlcuindo as informações de geolocalização (ponto, endereço, tipo, classe, latitude e longitude

df_estadio_estadio['ponto'] = df_estadio_estadio['location'].apply(lambda x : x.point if x else None)
df_estadio_estadio['endereco'] = df_estadio_estadio['location'].apply(lambda x : x.address if x else None)
df_estadio_estadio['tipo'] = df_estadio_estadio['location'].apply(lambda x : x.raw['type'] if x else None)
df_estadio_estadio['classe'] = df_estadio_estadio['location'].apply(lambda x : x.raw['class'] if x else None)
df_estadio_estadio['latitude'] = df_estadio_estadio['location'].apply(lambda x : x.latitude if x else None)
df_estadio_estadio['longitude'] = df_estadio_estadio['location'].apply(lambda x : x.longitude if x else None)

In [34]:
df_estadio_estadio.head()

,estadio,local,location,ponto,endereco,tipo,classe,latitude,longitude
0,"Estadio Vila Belmiro, Santos, SP.","Vila Belmiro, Santos, SP.","(Estádio Urbano Caldeira, s/n, Rua Princesa Is...","23 57m 4.01652s S, 46 20m 20.3686s W","Estádio Urbano Caldeira, s/n, Rua Princesa Isa...",stadium,leisure,-23.951116,-46.338991
1,"Estadio Independência, Belo Horizonte, MG.","Independência, Belo Horizonte, MG.",(Estádio Raimundo Sampaio (Arena Independência...,"19 54m 30.0694s S, 43 55m 4.00071s W",Estádio Raimundo Sampaio (Arena Independência)...,stadium,leisure,-19.908353,-43.917778
2,"Estadio Estádio da Serrinha, Goiânia, GO.","Estádio da Serrinha, Goiânia, GO.",None,None,None,None,None,NaN,NaN
3,"Estadio Mangueirão, Belém, PA.","Mangueirão, Belém, PA.",(Estádio Estadual Jornalista Edgar Augusto Pro...,"1 22m 52.0918s S, 48 26m 41.3752s W",Estádio Estadual Jornalista Edgar Augusto Proe...,yes,building,-1.381137,-48.444826
4,"Estadio Centenário, Montevidéu, Uruguai.","Centenário, Montevidéu, Uruguai.","(Estadio Centenario, Avenida Federico R. Vidie...","34 53m 40.3413s S, 56 9m 9.66274s W","Estadio Centenario, Avenida Federico R. Vidiel...",stadium,leisure,-34.894539,-56.152684


In [37]:
df_estadio_estadio.drop(columns=['estadio'], inplace=True)

In [38]:
# aqui estão os locais relacionados a esporte

df_estadio_estadio_esporte = df_estadio_estadio.query('tipo == "stadium" or tipo == "sports_centre" or tipo == "pitch" or tipo == "sports_hall" or tipo == "sports"')

In [39]:
todos_estadios = pd.concat([df_estadio_estadio_esporte, df_estadios_esporte]).reset_index(drop=True)

In [40]:
# jogos com os estádios com as informações de esporte

jogos_localizacao = jogos_completos.merge(todos_estadios, how='outer')

# Mapa

In [41]:
jogos_localizacao.head()

,campeonato,data,mandante,gol mandante,gol visitante,visitante,local,location,ponto,endereco,tipo,classe,latitude,longitude
0,Brasileirão 2023,"02 de julho de 2023, 11:00",Corinthians,0,1,Red Bull Bragantino,"Neo Química Arena, São Paulo, SP.","(Neo Química Arena, 111, Caminho Areita Calfat...","23 32m 43.0562s S, 46 28m 27.4312s W","Neo Química Arena, 111, Caminho Areita Calfat ...",stadium,leisure,-23.545293,-46.474286
1,Libertadores 2023,"28 de junho de 2023, 21:30",Corinthians,3,0,Liverpool,"Neo Química Arena, São Paulo, SP.","(Neo Química Arena, 111, Caminho Areita Calfat...","23 32m 43.0562s S, 46 28m 27.4312s W","Neo Química Arena, 111, Caminho Areita Calfat ...",stadium,leisure,-23.545293,-46.474286
2,Brasileirão 2023,"10 de junho de 2023, 18:30",Corinthians,1,1,Cuiabá,"Neo Química Arena, São Paulo, SP.","(Neo Química Arena, 111, Caminho Areita Calfat...","23 32m 43.0562s S, 46 28m 27.4312s W","Neo Química Arena, 111, Caminho Areita Calfat ...",stadium,leisure,-23.545293,-46.474286
3,Copa do Brasil 2023,"31 de maio de 2023, 21:30",Corinthians,2,0,Atlético-MG,"Neo Química Arena, São Paulo, SP.","(Neo Química Arena, 111, Caminho Areita Calfat...","23 32m 43.0562s S, 46 28m 27.4312s W","Neo Química Arena, 111, Caminho Areita Calfat ...",stadium,leisure,-23.545293,-46.474286
4,Brasileirão 2023,"28 de maio de 2023, 16:00",Corinthians,2,0,Fluminense,"Neo Química Arena, São Paulo, SP.","(Neo Química Arena, 111, Caminho Areita Calfat...","23 32m 43.0562s S, 46 28m 27.4312s W","Neo Química Arena, 111, Caminho Areita Calfat ...",stadium,leisure,-23.545293,-46.474286


In [42]:
jogos_localizacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6093 entries, 0 to 6092
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   campeonato     6093 non-null   object 
 1   data           6093 non-null   object 
 2   mandante       6093 non-null   object 
 3   gol mandante   6093 non-null   object 
 4   gol visitante  6093 non-null   object 
 5   visitante      6093 non-null   object 
 6   local          6093 non-null   object 
 7   location       5308 non-null   object 
 8   ponto          5308 non-null   object 
 9   endereco       5308 non-null   object 
 10  tipo           5308 non-null   object 
 11  classe         5308 non-null   object 
 12  latitude       5308 non-null   float64
 13  longitude      5308 non-null   float64
dtypes: float64(2), object(12)
memory usage: 666.5+ KB


In [47]:
jogos_localizacao.to_csv('jogos_localizacao_20230703.csv', index=False)

In [48]:
jogos_localizacao = jogos_localizacao.dropna()

In [52]:
jogos_localizacao['ponto'] = jogos_localizacao.apply(lambda row : Point(row['longitude'], row['latitude']), axis=1)

/tmp/ipykernel_9032/778713376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_localizacao['ponto'] = jogos_localizacao.apply(lambda row : Point(row['longitude'], row['latitude']), axis=1)


In [53]:
gdf_jogos_localizacao = gpd.GeoDataFrame(jogos_localizacao, geometry='ponto')

In [55]:
gdf_jogos_localizacao.crs = 'EPSG:4326'

In [57]:
fmap = folium.Map()
mc = FastMarkerCluster(gdf_jogos_localizacao[['latitude', 'longitude']])
fmap.add_child(mc)
fmap.save('mapa_corinthians.html')

# LIXO

In [ ]:
local.drop(columns=['mandante', 'estadio'], inplace=True)

In [ ]:
local_estadio = local.query('tipo == "stadium" or tipo == "pitch" or tipo == "sports_centre"')

In [ ]:
todos_estadios = pd.concat([local_estadio ,df_estadios_esporte]).reset_index(drop=True)

In [ ]:
# jogos com os estádios com as informações de esporte

jogos_localizacao = jogos_completos.merge(todos_estadios, how='outer')

In [ ]:
jogos_1 = jogos_localizacao[jogos_localizacao['location'].notna()]

In [ ]:
jogos_1.to_csv('corinthians_provisorio.csv')

In [ ]:
type(jogos_1['location'][0])

In [ ]:
mapa = gpd.GeoDataFrame(jogos_1)

In [ ]:
mapa['ponto']

In [ ]:
mapa = gpd.read_file()

In [ ]:
# arquivos com estádios que não foram incluidos

local = jogos_localizacao[jogos_localizacao['location'].isna()][['local', 'mandante']].groupby(by='local').count().sort_values(by='mandante', ascending=False).reset_index()

In [ ]:
# criando a coluna com o estadio + o nome do local

local['estadio'] = local['local']
local['estadio'] = 'estadio ' + local['estadio']

In [ ]:
# aplicando a geolocalização
local['location'] = local['estadio'].apply(geocode)

In [ ]:
# inlcuindo as informações de geolocalização (ponto, endereço, tipo, classe, latitude e longitude

local['ponto'] = local['location'].apply(lambda x : x.point if x else None)
local['endereco'] = local['location'].apply(lambda x : x.address if x else None)
local['tipo'] = local['location'].apply(lambda x : x.raw['type'] if x else None)
local['classe'] = local['location'].apply(lambda x : x.raw['class'] if x else None)
local['latitude'] = local['location'].apply(lambda x : x.latitude if x else None)
local['longitude'] = local['location'].apply(lambda x : x.longitude if x else None)

In [ ]:
local_estadio = local.query('tipo == "stadium" or tipo == "pitch" or tipo == "sports_centre"')

In [ ]:
local_estadio = local_estadio.drop(columns=['mandante', 'estadio'])

In [ ]:
jogos_localizacao.merge(local_estadio, how='outer')

In [ ]:
local

In [ ]:
local.query('tipo == "stadium" or tipo == "pitch" or tipo == "sports_centre"')

In [ ]:
df = []
erros = []


for link_timao in nao_encontrados['link']:
        
        # tenta pegar as informações
        try:
            
            # entra no link
            driver.get(link_timao)
            
            # pega o html e processa
            resultado = driver.page_source
            soup_resultado = BeautifulSoup(resultado, 'html.parser')
            
            # pega as informações da caixa
            h2 = soup_resultado.find(id="container_placar").find_all('h2')[1]

            # pega o campeonato
            campeonato = h2.text.strip().split('\n')[0]
            
            # pega a data
            data = h2.text.strip().split('\n')[1]
            
            # pega o placar e divide entre mandante e visitante
            placar_completo = h2.text.strip().split('\n')[2].strip()
            placar = placar_completo.split('x')
            
            # pega informações do time mandante (nome e gols)
            marcacao_mandante = placar[0].strip().find(' ', -1)
            mandante = placar[0].strip()[:marcacao_mandante].strip()
            gol_mandante = placar[0].strip()[marcacao_mandante:].strip()

            # pega informações do time visitante (nome e gols)
            marcacao_visitante = placar[1].strip().find(' ')
            visitante = placar[1].strip()[marcacao_visitante:].strip()
            gol_visitante = placar[1].strip()[:marcacao_visitante].strip()

            # pega o local
            local = h2.text.strip().split('\n')[4]
            campeonato = 
            
            # coloca as informações em uma lista
            df.append([campeonato, data, mandante, gol_mandante, gol_visitante, visitante, local])

        except:
            
            # caso tenha erro, pega o número da página, o link da partida e a partida
            erros.append([link_timao])
            
print('FIM')     

In [ ]:
def limpar_mandante(x):
    partida = x.split(' x ')
    marca = partida[0].find(' ', -1)
    return partida[0][:marca].strip()

In [ ]:
def limpar_visitante(x):
    partida = x.split(' x ')
    marca = partida[1].find(' ')
    return partida[1][marca:].strip()

In [ ]:
nao_encontrados['visitante'] = nao_encontrados['partida'].apply(limpar_visitante)
nao_encontrados['mandante'] = nao_encontrados['partida'].apply(limpar_mandante)

In [ ]:
nao_encontrados['link'][0]

In [ ]:
jogos_corinthians_masculino = pd.read_csv('jogos corinthians masculino.csv')

In [ ]:
jogos_corinthians_masculino.rename(columns={'data' :'campeonato', 'mandante': 'data', 'gol mandante': 'mandante', 'gol visitante' : 'gol mandante', 'visitante': 'gol visitante', 'estadio' : 'visitante', 'campeonato' : 'local'}, inplace=True)

In [ ]:
jogos_corinthians_masculino

In [ ]:
nao_encontrados['partida'].str.split(' x ', expand=True)[1][0][marca:].strip()

In [ ]:
# transformando apenas os estádios em um data frame

df_estadios = jogos_corinthians_masculino[['estadio']].drop_duplicates().reset_index(drop=True)

In [ ]:
# visualização dos cinco primeiros resultados

df_estadios.head()

In [ ]:
# instanciando a biblioteca que tratá as informações de geolocalização
locator = Nominatim(user_agent='MyGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

# aplicando a geolocalização
df_estadios['location'] = df_estadios['estadio'].apply(geocode)

In [ ]:
# inlcuindo as informações de geolocalização (ponto, endereço, tipo, classe, latitude e longitude

df_estadios['ponto'] = df_estadios['location'].apply(lambda x : x.point if x else None)
df_estadios['endereco'] = df_estadios['location'].apply(lambda x : x.address if x else None)
df_estadios['tipo'] = df_estadios['location'].apply(lambda x : x.raw['type'] if x else None)
df_estadios['classe'] = df_estadios['location'].apply(lambda x : x.raw['class'] if x else None)
df_estadios['latitude'] = df_estadios['location'].apply(lambda x : x.latitude if x else None)
df_estadios['longitude'] = df_estadios['location'].apply(lambda x : x.longitude if x else None)

In [ ]:
# inlcuindo as informações de geolocalização (latitude e longitude)

df_estadios['latitude'] = df_estadios['location'].apply(lambda x : x.latitude if x else None)
df_estadios['longitude'] = df_estadios['location'].apply(lambda x : x.longitude if x else None)

In [ ]:
# verificando quantos foram estádios

df_estadios.query('tipo == "stadium"')

In [ ]:
# buscando pelo nome do local + estádio

df_estadio_estadio['estadio '] = df_estadios.query('tipo != "stadium"')[['estadio']].apply(lambda x : 'Estadio ' + x)
df_estadio_estadio.reset_index(drop=True, inplace=True)
df_estadio_estadio['nome original'] = df_estadios.query('tipo != "stadium"')['estadio'].reset_index(drop=True)

# instanciando a biblioteca que tratá as informações de geolocalização
locator = Nominatim(user_agent='MyGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

# aplicando a geolocalização
df_estadio_estadio['location'] = df_estadio_estadio['estadio'].apply(geocode)

In [ ]:
# instanciando a biblioteca que tratá as informações de geolocalização
locator = Nominatim(user_agent='MyGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

# aplicando a geolocalização
df_estadio_estadio['location'] = df_estadio_estadio['estadio'].apply(geocode)

In [ ]:
# inlcuindo as informações de geolocalização (ponto, endereço, tipo, classe)

df_estadio_estadio['ponto'] = df_estadio_estadio['location'].apply(lambda x : x.point if x else None)
df_estadio_estadio['endereco'] = df_estadio_estadio['location'].apply(lambda x : x.address if x else None)
df_estadio_estadio['tipo'] = df_estadio_estadio['location'].apply(lambda x : x.raw['type'] if x else None)
df_estadio_estadio['classe'] = df_estadio_estadio['location'].apply(lambda x : x.raw['class'] if x else None)

In [ ]:
# inlcuindo as informações de geolocalização (latitude e longitude)

df_estadio_estadio['latitude'] = df_estadio_estadio['location'].apply(lambda x : x.latitude if x else None)
df_estadio_estadio['longitude'] = df_estadio_estadio['location'].apply(lambda x : x.longitude if x else None)

In [ ]:
df_1 = limpar(df_estadio_estadio.query('tipo == "pitch" or tipo == "stadium"'))

In [ ]:
df_2 = df_estadio_estadio.drop(columns=['estadio']).query('tipo == "pitch" or tipo == "stadium"').rename(columns={'nome original' : 'estadio'})

In [ ]:
df_final = pd.concat([df_1, df_2])

In [ ]:
df_final = df_final.drop_duplicates(subset='estadio')

In [ ]:
df_estadio_estadio.query('tipo != "pitch" & tipo != "stadium"')['tipo'].drop_duplicates()

In [ ]:
df_estadio_estadio.query('tipo == "yes"')

In [ ]:
df_3 = df_estadio_estadio[df_estadio_estadio['nome original'] == 'Mangueirão'].drop(columns=['estadio']).rename(columns={'nome original' : 'estadio'})

In [ ]:
df_final = pd.concat([df_final, df_3])

In [ ]:
df_estadio_estadio.query('tipo == "park"')

In [ ]:
df_4 = df_estadio_estadio.query('tipo == "sports_centre" & estadio != "Estadio Nacional" & estadio != "Estadio Eucaliptos" &  estadio != "Estadio Rua Campos Sales"')

In [ ]:
df_4 = df_4.drop(columns=['estadio']).rename(columns={'nome original' : 'estadio'})

In [ ]:
df_final = pd.concat([df_final, df_4]) 

In [ ]:
def limpar(df):
    return df.drop(columns=['estadio']).rename(columns={'nome original' : 'estadio'})

In [ ]:
df_5  = limpar(df_estadio_estadio.query('tipo == "service"'))

In [ ]:
df_final = pd.concat([df_final, df_5])

In [ ]:
df_estadio_estadio.query('tipo == "recreation_ground"')

In [ ]:
df_estadio_estadio['location'][67]

In [ ]:
jogos_corinthians_masculino.query('estadio == "Pueblo Nuevo"')

In [ ]:
endereços_novos = {"Jonas Duarte" : "Av. Brasil Sul, S/N - Jardim Ana Paula, Anápolis - GO",
                  "Beira Rio" : "Av. Padre Cacique, 891 - Praia de Belas, Porto Alegre ",
                  "Eucaliptos" : "R. Silveiro, 300 - Menino Deus, Porto Alegre",
                  "Rua Campos Sales": "Rua Campos Sales - Tijuca - Rio de Janeiro - Rio de Janeiro"
                  "":""}

In [ ]:
len(jogos_corinthians_masculino)

In [ ]:
jogos_estadio = jogos_corinthians_masculino.merge(df_final)

In [ ]:
jogos_estadio[jogos_estadio['estadio'] == 'Neo Quimica Arena']

In [ ]:
jogos_corinthians_masculino

In [ ]:
mask = df_final['estadio'] == 'Neo Química Arena'

In [ ]:
df_final[mask]

In [ ]:
link = jogo.find('a').get('href')

In [ ]:

'https://www.meutimao.com.br' + link

In [ ]:
jogos_lista.find_all('a').get('class')

In [ ]:
df = []
erros = []
page_inicial = 1
page_final = 205

for num in range(page_inicial, page_final):
    
    # entra no site e pega todos resultados dos jogos
    driver.get(f'https://www.meutimao.com.br/resultados-dos-jogos-do-corinthians/?pag={num}')
    site = driver.page_source
    soup = BeautifulSoup(site, 'html.parser')
    jogos_lista = soup.find(id='jogos_lista')
    jogos = jogos_lista.find_all('li')
    
    # looping de mineração de cada jogo 
    link_partida = {}
    
    todos_links = []
    for jogo in jogos:
        link = jogo.find('a').get('href')
        todos_links.append('https://www.meutimao.com.br' + link)
        
        # marcação do que deve ser pego no texto da partida
        marcacao_partida = jogo.find('a').text.find(':')
        
        # texto da partida
        partida = jogo.find('a').text[:marcacao_partida].replace('\n', '').split('x')
        link_partida['https://www.meutimao.com.br' + link] = jogo.find('a').text[:marcacao_partida].replace('\n', '')
    
    for link_timao in todos_links:
        try:
            driver.get(link_timao)

            resultado = driver.page_source
            soup_resultado = BeautifulSoup(resultado, 'html.parser')

            h2 = soup_resultado.find(id="container_placar").find_all('h2')[1]

            campeonato = h2.text.strip().split('\n')[0]
            data = h2.text.strip().split('\n')[1]
            placar_completo = h2.text.strip().split('\n')[2].strip()
            
            placar = placar_completo.split('x')
            
            # pega informações do time mandante (nome e gols)
            marcacao_mandante = placar[0].strip().find(' ', -1)
            mandante = placar[0].strip()[:marcacao_mandante].strip()
            gol_mandante = placar[0].strip()[marcacao_mandante:].strip()

            # pega informações do time visitante (nome e gols)
            marcacao_visitante = placar[1].strip().find(' ')
            visitante = placar[1].strip()[marcacao_visitante:].strip()
            gol_visitante = placar[1].strip()[:marcacao_visitante].strip()

            # pega o local
            local = h2.text.strip().split('\n')[4]
            df.append([campeonato, data, mandante, gol_mandante, gol_visitante, visitante, local])

        except:
            erros.append([num, link_timao, link_partida['https://www.meutimao.com.br' + link]])
            
print('FIM')     

In [ ]:
df = pd.DataFrame(df, columns=[0, 1, 2, 3, 4, 5, 6])

In [ ]:
jogo.find('a').text[:marcacao_partida].replace('\n', '')

In [ ]:

resultado = driver.page_source
soup_resultado = BeautifulSoup(resultado, 'html.parser')

h2 = soup_resultado.find(id="container_placar").find_all('h2')[1]

campeonato = h2.text.strip().split('\n')[0]
data = h2.text.strip().split('\n')[1]
placar_completo = h2.text.strip().split('\n')[2].strip()

# pega informações do time mandante (nome e gols)
marcacao_mandante = placar[0].strip().find(' ', -1)
mandante = placar[0].strip()[:marcacao_mandante].strip()
gol_mandante = placar[0].strip()[marcacao_mandante:].strip()

# pega informações do time visitante (nome e gols)
marcacao_visitante = placar[1].strip().find(' ')
visitante = placar[1].strip()[marcacao_visitante:].strip()
gol_visitante = placar[1].strip()[:marcacao_visitante].strip()

# pega o local
local = h2.text.strip().split('\n')[4]

In [ ]:
link_partida['https://www.meutimao.com.br/jogo/5926/brasileirao_2021/sport-1-x-0-corinthians']

In [ ]:
placar_completo = h2.text.strip().split('\n')[2].strip()
placar_completo

In [ ]:
placar = placar_completo.split('x')

In [ ]:
# pega informações do time mandante (nome e gols)
marcacao_mandante = placar[0].strip().find(' ', -1)
mandante = placar[0].strip()[:marcacao_mandante].strip()
gol_mandante = placar[0].strip()[marcacao_mandante:].strip()
print(mandante, gol_mandante)

In [ ]:
        
# pega informações do time visitante (nome e gols)
marcacao_visitante = placar[1].strip().find(' ')
visitante = placar[1].strip()[marcacao_visitante:].strip()
gol_visitante = placar[1].strip()[:marcacao_visitante].strip()


In [ ]:
local = h2.text.strip().split('\n')[4]
local